# 1. 
* 用pytorch做cnn模型
        

In [1]:
import jieba 
import gensim
import numpy as np
import sklearn
import torch
from torch import nn
from sklearn.metrics import f1_score
import torch.nn.functional as F
from batch_size import datasets
from torch.utils.data import Dataset, DataLoader
from preprocess import pre_train
import config as c
from torch.autograd import Variable

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
word2vec_model=gensim.models.KeyedVectors.load_word2vec_format(c.word2vec_file,binary=True)
load_word=pre_train()
data=load_word.load_word()

* 首先知道需要的矩阵大小初始化一个zeros

In [3]:
def get_data_cnn(idx):
    '''
    就是把数据传入
    N,c,W,h
    '''
    batch=np.zeros((len(idx),2,c.max_words,c.word_dim))
    
    for index,bat in enumerate(idx):
        #第一个问题的单词
        batch[index][0]=transform_word_vector_cnn(data['q1'][bat])
        #第二个单词的问题
        batch[index][1]=transform_word_vector_cnn(data['q2'][bat])  
        
    target=np.array([data['target'][i] for i in idx])    
    return batch,target 

def transform_word_vector_cnn(word_lists):
    '''
    分好词的句子的列表
    '''
    re=np.zeros((c.max_words,128))
    insert_vec=0
    for word in word_lists:
        if insert_vec<re.shape[0]:
            if word in word2vec_model.vocab:
                re[insert_vec]=word2vec_model[word]
                insert_vec+=1
    return re

In [4]:
dataset=datasets()
datas=DataLoader(dataset,batch_size=64,shuffle=True)

### 2. cnn
* 一句话可以描述为，最多为15个单词， 每个单词维度128===15*128
* 然后每个样本是两句话。
* 多个样本

* Dropout2d

In [10]:
m = nn.Dropout2d(p=0.2)
input = torch.randn(20, 16, 32, 32)
output = m(input)
output.shape

torch.Size([20, 16, 32, 32])

#### 感受尺寸变化

In [6]:
nn.Conv2d?

In [11]:
#产生数据
x=torch.randn(20,2,12,64)
x.shape

torch.Size([20, 2, 12, 64])

In [12]:
#第一个卷积，磁化
conv1=nn.Conv2d(2,64,kernel_size=(3,15),padding=2)
x=F.relu(F.max_pool2d(conv1(x),(2,)))
x.shape

torch.Size([20, 64, 7, 27])

In [22]:
F.max_pool2d?

In [13]:
#第二个卷积，磁化
conv2=nn.Conv2d(64,12,kernel_size=(3,5),padding=2)#
drop=nn.Dropout2d()
x = F.relu(F.max_pool2d(drop(conv2(x)), (2,)))
x.shape

torch.Size([20, 12, 4, 13])

In [14]:
12*13*4

624

* 模型定义

In [33]:
class cnn(nn.Module):
    def __init__(self):
        '''
        双通道输入类别预测。
        '''
        super(cnn,self).__init__()
        self.conv1=nn.Conv2d(2,64,kernel_size=(3,15),padding=2)#
        self.conv2=nn.Conv2d(64,12,kernel_size=(3,5),padding=2)#
        self.conv2_drop=nn.Dropout2d()
        self.fc1=nn.Linear(624,100)
        self.fc2=nn.Linear(100,2)
        self.soft=nn.Softmax(dim=1)
    def forward(self,x):
        x=F.tanh(F.max_pool2d(self.conv1(x),(2,)))
        x = F.tanh(F.max_pool2d(self.conv2_drop(self.conv2(x)), (2,)))
        x=x.view(-1,624)
        x=F.relu(self.fc1(x))
        x=F.dropout(x,training=self.training)
        x=self.fc2(x)
        x=self.soft(x)
        return x

In [42]:
class cnn2(nn.Module):
    def __init__(self):
        '''
        双通道输入类别预测。
        '''
        super(cnn2,self).__init__()
        self.conv1=nn.Conv2d(2,12,kernel_size=5,padding=2)#
        self.conv2=nn.Conv2d(12,2,kernel_size=5,padding=2)#
        self.fc1=nn.Linear(128,2)

    def forward(self,x):
        x=F.relu(F.max_pool2d(self.conv1(x),2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x=x.view(-1,128)
        x=self.fc1(x)
        return x

* 训练

In [34]:
#损失函数，
model=cnn()
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adadelta(model.parameters(),lr=5e-3)

In [35]:
epochs=100
for epoch in range(epochs):
    for i,idx in enumerate(datas):
        train_x,train_y=get_data_cnn(idx.numpy())
        inputs=Variable(torch.from_numpy(train_x).float())
        labels=Variable(torch.tensor(train_y).long())
        y_pred=model(inputs)        
        loss=loss_fn(y_pred,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if  i%1000==0:
            print(epoch,i,loss.item()) 

0 0 0.6858846545219421
1 0 0.7033414840698242
2 0 0.6924723982810974
3 0 0.6982866525650024
4 0 0.675123393535614
5 0 0.7092145681381226
6 0 0.6944957375526428
7 0 0.697792112827301
8 0 0.7020511627197266
9 0 0.6958983540534973
10 0 0.684502363204956
11 0 0.6898574233055115
12 0 0.6931677460670471
13 0 0.6949463486671448
14 0 0.6914944648742676
15 0 0.6928578019142151


KeyboardInterrupt: 


* 评估模型

In [ ]:
x,y=get_data_cnn(list(np.arange(len(datas['index']))))

In [ ]:
model.eval()
with torch.no_grad():
    y=model(inputs)

y_pred=y.argmax(dim=1).numpy()    

print('f1_score',i,f1_score(train_y,y_pred))    

* 保存模型

* load 模型

* 训练的结果导致需要一个阈值来保证最大化
        如何求阈值？

* 将两列合并之后再训练
   
       1.1 lightgbm 简单
       1.2 线性呢？
       1.3 torch神经网络？

### 6.欠采样